In [1]:
#Twitter Sentiments Classification

In [2]:
import pandas as pd
import numpy as np
import spacy
from tqdm import tqdm
import re
import time
import pickle
pd.set_option('display.max_colwidth', 200)

C:\ProgramData\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
C:\ProgramData\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


In [3]:
# read data
train = pd.read_csv("train_2kmZucJ.csv")
test = pd.read_csv("test_oJQbWVk.csv")

train.shape, test.shape

((7920, 3), (1953, 2))

In [4]:
train['label'].value_counts(normalize = True)

0    0.744192
1    0.255808
Name: label, dtype: float64

In [5]:
train.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1MfQV #android #apps #beautiful #cute #health #igers #iphoneonly #iphonesia #iphone
1,2,0,Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias… http://instagram.com/p/YGEt5JC6JM/
2,3,0,We love this! Would you go? #talk #makememories #unplug #relax #iphone #smartphone #wifi #connect... http://fb.me/6N3LsUpCu
3,4,0,I'm wired I know I'm George I was made that way ;) #iphone #cute #daventry #home http://instagr.am/p/Li_5_ujS4k/
4,5,1,What amazing service! Apple won't even talk to me about a question I have unless I pay them $19.95 for their stupid support!


In [6]:
# remove URL's from train and test
train['clean_tweet'] = train['tweet'].apply(lambda x: re.sub(r'http\S+', '', x))

test['clean_tweet'] = test['tweet'].apply(lambda x: re.sub(r'http\S+', '', x))

In [7]:
# remove punctuation marks
punctuation = '!"#$%&()*+-/:;<=>?@[\\]^_`{|}~'

train['clean_tweet'] = train['clean_tweet'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))
test['clean_tweet'] = test['clean_tweet'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))

# convert text to lowercase
train['clean_tweet'] = train['clean_tweet'].str.lower()
test['clean_tweet'] = test['clean_tweet'].str.lower()

# remove numbers
train['clean_tweet'] = train['clean_tweet'].str.replace("[0-9]", " ")
test['clean_tweet'] = test['clean_tweet'].str.replace("[0-9]", " ")

# remove whitespaces
train['clean_tweet'] = train['clean_tweet'].apply(lambda x:' '.join(x.split()))
test['clean_tweet'] = test['clean_tweet'].apply(lambda x: ' '.join(x.split()))

In [9]:
# import spaCy's language model
nlp = spacy.load('en', disable=['parser', 'ner'])

# function to lemmatize text
def lemmatization(texts):
    output = []
    for i in texts:
        s = [token.lemma_ for token in nlp(i)]
        output.append(' '.join(s))
    return output

C:\ProgramData\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
C:\ProgramData\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


In [10]:
train['clean_tweet'] = lemmatization(train['clean_tweet'])
test['clean_tweet'] = lemmatization(test['clean_tweet'])

In [11]:
train.sample(10)

,id,label,tweet,clean_tweet
1466,1467,0,RT @littlewingman: The first & only adult-rated pickup line app for the #iPhone! http://www.LittleWingman.com #$&@*# #romance,rt littlewingman the first only adultrat pickup line app for the iphone romance
618,619,1,"I want the new iPad and the new iPhone 5S.. but wait, where's the money #apple","i want the new ipad and the new iphone s .. but wait , where be the money apple"
5050,5051,0,@GamersComplain that's why you have to get a #samsung S5. You can bend it or throw it at the wall and it still works #blessedindeed,gamerscomplain that be why -PRON- have to get a samsung s . -PRON- can bend -PRON- or throw -PRON- at the wall and -PRON- still work blessedinde
6353,6354,0,"You might not buy the Facebook phone, but your kids will and that’s what matters http://dlvr.it/1dX9dG #google #Apple","-PRON- may not buy the facebook phone , but -PRON- kid will and that ’ what matter google apple"
4558,4559,1,thanks again #apple for updating my iphone without my permission.. it runs so very SLOW now... #iphone5 #notcool,thank again apple for update -PRON- iphone without -PRON- permission .. -PRON- run so very slow now ... iphone notcool
3742,3743,0,SHINE BRIGHT LIKE A DIAMOND! #rihanna #music #tune #cute #lyrics #itunes #iphone #iphoneisa http://instagr.am/p/QGE18iln5d/,shine bright like a diamond rihanna music tune cute lyric itun iphone iphoneisa
4991,4992,0,NEW CASE *-* #case #samsung #galaxy #ace #2 #blue #me #peace #swag #lovatic #belieber #smiler… http://instagram.com/p/aS8hIYQX5W/,new case case samsung galaxy ace blue -PRON- peace swag lovatic belieber smiler …
3038,3039,0,"- 004 - Happy Ied Mubarrak from One Day, One Chord Progression: i ii7b5. Follow to get hundreds of chord progressions. #music #musically #musicproducer #musicians #piano #guitar #guitarist #song #...","happy i mubarrak from one day , one chord progression i ii b . follow to get hundred of chord progression . music musically musicproducer musician piano guitar guitarist song songwriter musictheor..."
6133,6134,1,@anthonyli is a traitor. i don't plan on speaking to you on Friday. #hateiphones. . #bitter.,anthonyli be a traitor . i do not plan on speak to -PRON- on friday . hateiphone . . bitter .
6644,6645,1,"Fun Fact: If you get the new iPhone cable wet, it no longer works. #apple #technology #water","fun fact if -PRON- get the new iphone cable wet , -PRON- no longer work . apple technology water"


You are using pip version 9.0.3, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [14]:
import tensorflow_hub as hub
import tensorflow as tf

elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
W0313 15:50:31.239981 20252 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [15]:
# just a random sentence
x = ["Roasted ants are a popular snack in Columbia"]

# Extract ELMo features 
embeddings = elmo(x, signature="default", as_dict=True)["elmo"]

embeddings.shape

TensorShape([Dimension(1), Dimension(8), Dimension(1024)])

In [16]:
def elmo_vectors(x):
  embeddings = elmo(x.tolist(), signature="default", as_dict=True)["elmo"]

  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # return average of ELMo features
    return sess.run(tf.reduce_mean(embeddings,1))

In [17]:
list_train = [train[i:i+100] for i in range(0,train.shape[0],100)]
list_test = [test[i:i+100] for i in range(0,test.shape[0],100)]

In [18]:
# Extract ELMo embeddings
elmo_train = [elmo_vectors(x['clean_tweet']) for x in list_train]
elmo_test = [elmo_vectors(x['clean_tweet']) for x in list_test]

In [19]:
elmo_train_new = np.concatenate(elmo_train, axis = 0)
elmo_test_new = np.concatenate(elmo_test, axis = 0)

In [20]:
# save elmo_train_new
pickle_out = open("elmo_train_03032019.pickle","wb")
pickle.dump(elmo_train_new, pickle_out)
pickle_out.close()

# save elmo_test_new
pickle_out = open("elmo_test_03032019.pickle","wb")
pickle.dump(elmo_test_new, pickle_out)
pickle_out.close()

In [21]:
# load elmo_train_new
pickle_in = open("elmo_train_03032019.pickle", "rb")
elmo_train_new = pickle.load(pickle_in)

# load elmo_train_new
pickle_in = open("elmo_test_03032019.pickle", "rb")
elmo_test_new = pickle.load(pickle_in)

In [22]:
from sklearn.model_selection import train_test_split

xtrain, xvalid, ytrain, yvalid = train_test_split(elmo_train_new, 
                                                  train['label'],  
                                                  random_state=42, 
                                                  test_size=0.2)

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

lreg = LogisticRegression()
lreg.fit(xtrain, ytrain)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [24]:
preds_valid = lreg.predict(xvalid)

In [25]:
f1_score(yvalid, preds_valid)

0.7727272727272728

In [26]:
# make predictions on test set
preds_test = lreg.predict(elmo_test_new)

In [27]:
# prepare submission dataframe
sub = pd.DataFrame({'id':test['id'], 'label':preds_test})

# write predictions to a CSV file
sub.to_csv("sub_lreg.csv", index=False)